In [1]:
import kalshi_python
from credentials import EMAIL, PW

config = kalshi_python.Configuration()
config.host = "https://trading-api.kalshi.com/trade-api/v2"

kalshi_api = kalshi_python.ApiInstance(
email=EMAIL,
password=PW,
configuration=config,
)

In [2]:
TICKER = "VULTURES-24JAN12"

In [3]:
# get_last

market_response = kalshi_api.get_market(TICKER)
market = market_response.market
last_px = market.last_price

In [4]:
# get_orderbook

orderbook_response = kalshi_api.get_market_orderbook(TICKER)
orderbook = orderbook_response.orderbook
type(orderbook)
if len(orderbook.yes) > 0:
    best_bid = max(orderbook.yes, key=lambda x: x[0])[0]
else:
    best_bid = 0
if len(orderbook.no) > 0:
    best_offer = 100 - max(orderbook.no, key=lambda x: x[0])[0]
else:
    best_offer = 0
print(best_bid, best_offer)

21 27


In [10]:
# get trades

trades_response = kalshi_api.get_trades(ticker=TICKER)
trades = trades_response.trades
print(trades)

[{'trade_id': '0b2995f9-1b18-44f7-bcad-cc914ed7d570', 'ticker': 'VULTURES-24JAN12', 'count': 1, 'created_time': '2024-01-02T22:33:53.425159Z', 'yes_price': 21, 'no_price': 79, 'taker_side': 'no'}, {'trade_id': 'dbd9e8d4-536c-407d-9e15-d3f62d3194d9', 'ticker': 'VULTURES-24JAN12', 'count': 100, 'created_time': '2024-01-02T22:27:22.966654Z', 'yes_price': 26, 'no_price': 74, 'taker_side': 'yes'}, {'trade_id': '39a6b9cf-59b3-47a3-8806-8a60f37758c1', 'ticker': 'VULTURES-24JAN12', 'count': 45, 'created_time': '2024-01-02T20:52:29.978768Z', 'yes_price': 21, 'no_price': 79, 'taker_side': 'no'}, {'trade_id': 'cb17b688-1ee6-41cc-8aa8-a0f3bfdc1bf0', 'ticker': 'VULTURES-24JAN12', 'count': 5, 'created_time': '2024-01-02T19:32:41.063948Z', 'yes_price': 25, 'no_price': 75, 'taker_side': 'yes'}, {'trade_id': '4f2e7a38-78b5-4274-8f42-2b285a1ade1f', 'ticker': 'VULTURES-24JAN12', 'count': 5, 'created_time': '2024-01-02T19:31:53.964475Z', 'yes_price': 25, 'no_price': 75, 'taker_side': 'yes'}, {'trade_id': 

In [11]:
trades[-1]['created_time']

'2023-12-22T00:45:52.984146Z'

In [15]:
old_trades = trades[:50]
new_trades = trades[-50:]

def trade_diff(old_trades, new_trades):
    if len(old_trades) == 0:
        return new_trades
    last_ts = old_trades[-1]['created_time']
    trades = []
    for trade in new_trades:
        if trade['created_time'] > last_ts:
            trades.append(trade)
    return trades

trades = trade_diff(old_trades, new_trades)
len(trades)

46

In [6]:
# get_inventory

TICKER2 = "VULTURES-23JAN31"

inventory_response = kalshi_api.get_positions(ticker="INXZ-23DEC27-T4774.75")
# inventory_response.market_positions[0]
inventory_response
# position = 0
# for posn in inventory_response.market_positions:
#     print(posn)
#     if posn.ticker == TICKER:
#         position += posn.position
# position

{'cursor': None,
 'event_positions': [{'event_exposure': 39,
                      'event_ticker': 'INXZ-23DEC27',
                      'fees_paid': 3,
                      'realized_pnl': -119,
                      'resting_order_count': 2,
                      'total_cost': 3058}],
 'market_positions': [{'fees_paid': 3,
                       'market_exposure': 39,
                       'position': -1,
                       'realized_pnl': -119,
                       'resting_orders_count': 2,
                       'ticker': 'INXZ-23DEC27-T4774.75',
                       'total_traded': 3058}]}

In [48]:
# get open orders

get_orders_response = kalshi_api.get_orders(ticker=TICKER)
for order in get_orders_response.orders:
    if order['status'] == "resting":
        print(order)

In [43]:
# place_order
import uuid

yes_order = {'action': 'buy', 'type': 'limit', 'ticker': TICKER, 'count': 1, 'side': 'yes', 'yes_price': 28, 'client_order_id': str(uuid.uuid4())}
no_order = {'action': 'buy', 'type': 'limit', 'ticker': TICKER, 'count': 1, 'side': 'no', 'no_price': 77}

order_response = kalshi_api.create_order(yes_order)
order_response

{'order': {'action': 'buy',
           'client_order_id': '1f61ec83-9909-4fac-b933-b18f7f735839',
           'created_time': '2023-12-27T00:34:26.21883Z',
           'expiration_time': None,
           'no_price': 72,
           'order_id': '39a5f89f-3daf-4267-9921-e8119fc7b8c1',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'VULTURES-24JAN12',
           'type': 'limit',
           'user_id': '066cb11c-d233-4566-98dd-f53178e467a6',
           'yes_price': 28}}

In [47]:
# cancel order

ID = "39a5f89f-3daf-4267-9921-e8119fc7b8c1"

cancel_response = kalshi_api.cancel_order(ID)
cancel_response

{'order': {'action': 'buy',
           'client_order_id': '1f61ec83-9909-4fac-b933-b18f7f735839',
           'close_cancel_count': 0,
           'created_time': '2023-12-27T00:34:26.21883Z',
           'decrease_count': 1,
           'expiration_time': None,
           'fcc_cancel_count': 0,
           'last_update_time': '2023-12-27T00:35:18.703383Z',
           'maker_fill_count': 0,
           'no_price': 72,
           'order_id': '39a5f89f-3daf-4267-9921-e8119fc7b8c1',
           'place_count': 1,
           'queue_position': 0,
           'remaining_count': 0,
           'side': 'yes',
           'status': 'canceled',
           'taker_fees': 0,
           'taker_fill_cost': 0,
           'taker_fill_count': 0,
           'ticker': 'VULTURES-24JAN12',
           'type': 'limit',
           'user_id': '066cb11c-d233-4566-98dd-f53178e467a6',
           'yes_price': 28},
 'reduced_by': 1}